In [ ]:
import pickle

# 读取 coinbase.pkl 文件
with open('../coinbase.pkl', 'rb') as coinbase_file:
    coinbase_data = pickle.load(coinbase_file)

# 打印 coinbase_data 列表内容
for data in coinbase_data:
    print(data)

: 

: 

In [ ]:
import os
import json
import pickle
import requests
from datetime import datetime
from blockchain_parser.blockchain import Blockchain

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_transaction_data_to_json():
    blockchain_path = os.path.expanduser('~/.bitcoin/blocks')  # 指定比特幣 Core 區塊鏈數據的路徑
    blockchain = Blockchain(blockchain_path)

    start_date = datetime(2023, 6, 26, 0, 0)
    end_date = datetime(2023, 6, 26, 1, 1)

    transaction_count = 0
    directory_count = 0
    index_data = {}
    coinbase_data = {}

    address_directory = '../address'
    create_directory_if_not_exists(address_directory)

    for block in blockchain.get_unordered_blocks():
        # Check if the block timestamp is within the specified date range
        if start_date <= block.header.timestamp <= end_date:
            for tx in block.transactions:
                transaction_hash = tx.hash
                directory = str(directory_count).zfill(4)

                # Create address directory if it doesn't exist
                create_directory_if_not_exists(directory)
                
                # 因為不能是序列化 list 所以先用 tx.inputs[0] 的交易哈希值當地址
                # 未來這個要改成真的 address
                tx_id = tx.inputs[0]

                # 將 input_txid 轉換為字串
                input_str = str(tx_id)

                # 使用字串處理方法提取子字串
                start_index = input_str.index("(") + 1
                end_index = input_str.index(",", start_index)
                tx_id = input_str[start_index:end_index]

                # 使用 blockchain 網頁來提取 txid 的 input address
                api_url = f'https://blockchain.info/rawtx/{tx_id}'

                try:
                    response = requests.get(api_url)
                    if response.status_code == 200:
                        transaction_data = response.json()
                        input_addresses = [input['prev_out']['addr'] for input in transaction_data['inputs']]
                        if input_addresses:
                            # print("輸入地址：", input_addresses)
                            address = input_addresses[0]
                        else:
                            print("找不到輸入地址。")
                    else:
                        print("查詢交易失敗：", response.text)
                        address = "00000"

                except requests.exceptions.RequestException as e:
                    print("請求失敗：", str(e))


                print(address)

                address_directory_path = os.path.join(address_directory, f'{address}.json')
                if not os.path.exists(address_directory_path):
                    with open(address_directory_path, 'w') as address_file:
                        json.dump([address], address_file, indent=4)

                file_path = os.path.join(directory, f'{address}.json')

                with open(file_path, 'w') as file:
                    json.dump({
                        'timestamp': block.header.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                        'hash': tx.hash,
                        'inputs': [str(input_) for input_ in tx.inputs],
                        'outputs': [str(output) for output in tx.outputs],
                        'coinbase': tx.inputs[0]
                    }, file, indent=4, default=str)

                # Update index data
                index_data[transaction_hash] = directory

                # Update coinbase data
                coinbase_data[transaction_hash] = directory

                transaction_count += 1
                if transaction_count % 10000 == 0:
                    directory_count += 1

    # Append the "index" field to the index_data
    index_data = {"index": index_data}

    # Append the "coinbase" field to the coinbase_data
    coinbase_data = {"tx2blk": coinbase_data}

    # Save index data to index.json
    with open('index.json', 'w') as index_file:
        json.dump(index_data, index_file, indent=4)

    # Save coinbase data to coinbase.pkl
    with open('../coinbase.pkl', 'wb') as coinbase_file:
        pickle.dump(coinbase_data, coinbase_file)

save_transaction_data_to_json()


In [1]:
import os
import json
import pickle
import requests
from datetime import datetime
from blockchain_parser.blockchain import Blockchain

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_transaction_data_to_json():
    blockchain_path = os.path.expanduser('~/.bitcoin/blocks')  # 指定比特幣 Core 區塊鏈數據的路徑
    blockchain = Blockchain(blockchain_path)

    start_date = datetime(2023, 7, 1, 0, 0)
    end_date = datetime(2023, 7, 1, 1, 0)

    transaction_count = 0
    directory_count = 0
    index_data = {}
    coinbase_data = {}

    address_directory = '../address'
    create_directory_if_not_exists(address_directory)

    for block in blockchain.get_unordered_blocks():
        # Check if the block timestamp is within the specified date range
        if start_date <= block.header.timestamp <= end_date:
            for tx in block.transactions:
                transaction_hash = tx.hash
                directory = str(directory_count).zfill(4)

                # Create address directory if it doesn't exist
                create_directory_if_not_exists(directory)
                
                # 因為不能是序列化 list 所以先用 tx.inputs[0] 的交易哈希值當地址
                # 未來這個要改成真的 address
                tx_id = tx.inputs[0]

                # 將 input_txid 轉換為字串
                input_str = str(tx_id)

                # 使用字串處理方法提取子字串
                start_index = input_str.index("(") + 1
                end_index = input_str.index(",", start_index)
                tx_id = input_str[start_index:end_index]

                # 使用 blockchain 網頁來提取 txid 的 input address
                api_url = f'https://blockchain.info/rawtx/{tx_id}'

                try:
                    response = requests.get(api_url)
                    if response.status_code == 200:
                        transaction_data = response.json()
                        input_addresses = [input['prev_out']['addr'] for input in transaction_data['inputs']]
                        if input_addresses:
                            # print("輸入地址：", input_addresses)
                            address = input_addresses[0]
                        else:
                            print("找不到輸入地址。")
                    else:
                        print("查詢交易失敗：", response.text)
                        address = "00000"

                except requests.exceptions.RequestException as e:
                    print("請求失敗：", str(e))


                print(address)

                address_directory_path = os.path.join(address_directory, f'{address}.json')
                if not os.path.exists(address_directory_path):
                    with open(address_directory_path, 'w') as address_file:
                        json.dump([transaction_hash], address_file, indent=4)
                
                # 這個是 0000 下面的 tx id，所以要命名 transaction_hash.json
                file_path = os.path.join(directory, f'{transaction_hash}.json')

                with open(file_path, 'w') as file:
                    json.dump({
                        'timestamp': block.header.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                        'hash': tx.hash,
                        'inputs': [str(input_) for input_ in tx.inputs],
                        'outputs': [str(output) for output in tx.outputs],
                        'coinbase': tx.inputs[0]
                    }, file, indent=4, default=str)

                # Update index data
                index_data[transaction_hash] = directory

                # Update coinbase data
                coinbase_data[transaction_hash] = directory

                transaction_count += 1
                if transaction_count % 10000 == 0:
                    directory_count += 1

    # Append the "index" field to the index_data
    # index_data = {"index": index_data}

    # Append the "coinbase" field to the coinbase_data
    coinbase_data = {"tx2blk": coinbase_data}

    # Save index data to index.json
    with open('index.json', 'w') as index_file:
        json.dump(index_data, index_file, indent=4)

    # Save coinbase data to coinbase.pkl
    with open('../coinbase.pkl', 'wb') as coinbase_file:
        pickle.dump(coinbase_data, coinbase_file)

save_transaction_data_to_json()


查詢交易失敗： {"error":"not-found-or-invalid-arg","message":"Item not found or argument invalid"}
00000
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1FGiZB7K757EUixGcyeyME6Jp8qQZEiUUk
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
1HpTVtYFbMe9TbL3RTFxuJrhqBNbmbBk9F
1B1LL6LZjeoA7wp51vcJydMSQiEL6mUYp9
3DCGaDnfqqcDxcCarFmngqXJLp92afrczW
bc1p6n3ss06f29ryq9a9nqrxtpx7m4mhxefs58rwqg0jcamx3d2rtpvs5t9g07
bc1p6n3ss06f29ryq9a9nqrxtpx7m4mhxefs58rwqg0jcamx3d2rtpvs5t9g07
bc1q7cyrfmck2ffu2ud3rn5l5a8yv6f0chkp0zpemf
bc1q7cyrfmck2ffu2ud3rn5l5a8yv6f0chkp0zpemf
bc1q7cyrfmck2ffu2ud3rn5l5a8yv6f0chkp0zpemf
bc1q7cyrfmck2ffu2ud3rn5l5a8yv6f0chkp0zpemf
bc1q7cyrfmck2ffu2ud3rn5l5a8yv6f0chkp0zpemf
b